<a href="https://colab.research.google.com/github/keerthucit/EIP2Assignment/blob/master/Assignment5_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-pzy1lezi
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-pzy1lezi
  Running command git submodule update --init --recursive -q
  Stored in directory: /tmp/pip-ephem-wheel-cache-7h5bazoc/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [0]:
## Load the trained RESNET-18 model [ Change the input to CIFAR-10 input (32,32,3) instead of (224,224,3)]\n",
from classification_models.resnet import ResNet18, preprocess_input       
model = ResNet18((32, 32, 3), weights='imagenet', classes=10, include_top=False)


In [0]:
!pip install -q keras
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation,GlobalAveragePooling2D
n_classes = 10  
x = GlobalAveragePooling2D()(model.output)
# add a fully-connected layer
x = keras.layers.Dense(1024, activation='relu')(x)       
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[model.input], outputs=[output])

In [0]:
from keras.utils import np_utils
nb_classes = 10
# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.reshape(y_train.shape[0]) 
 # somehow y_train comes as a 2D nx1 matrix\n",
y_test = y_test.reshape(y_test.shape[0])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


In [0]:
## Pls check this link:- https://stackoverflow.com/questions/49259361/valueerror-decode-predictions-expects-a-batch-of-predictions-i-e-a-2d-array"

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#create data generator
datagen = ImageDataGenerator(
featurewise_center=False,  # set input mean to 0 over the dataset
samplewise_center=False,  # set each sample mean to 0
featurewise_std_normalization=False,  # divide inputs by std of the dataset
samplewise_std_normalization=False,  # divide each input by its std
zca_whitening=False,  # apply ZCA whitening
rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
horizontal_flip=True,  # randomly flip images
vertical_flip=False)  # randomly flip images   
# prepare iterator
it_train=datagen.flow(X_train, Y_train, batch_size=32)
it_test=datagen.flow(X_test, Y_test, batch_size=32)    
filepath="/content/gdrive/My Drive/Restnet18_PreTrained.hdf5"
mcp_save = keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', mode='min')     
steps = int(X_train.shape[0] / 32)
model_details = model.fit_generator(it_train, steps_per_epoch=steps, epochs=10, verbose=2, validation_data=it_test, validation_steps=len(it_test), callbacks=[mcp_save]),
# Score trained model.
scores = model.evaluate_generator(it_test, steps=len(it_test), verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

W0701 18:13:28.609982 140650101651328 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


In [0]:
predictions = model.predict([X_test])
y_class = predictions.argmax(axis=-1)

In [0]:
import numpy as n
correct_indices = np.nonzero(y_class == y_test)[0]
incorrect_indices = np.nonzero(y_class != y_test)[0]
print()
print(len(correct_indices),\" classified correctly\")
print(len(incorrect_indices),\" classified Incorrectly\")

In [0]:
examples_per_class = 8
y_test_pred= y_class
### class_name = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}
## Just for reference
import matplotlib.pyplot as plt
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
  plt.subplot(3, 3, i + 1)
  plt.imshow(X_test[correct].reshape(32,32,3), interpolation='none')
  plt.title(\"Predicted {}, Class {}\".format(y_class[correct], y_test[correct]))
plt.figure()
for i, incorrect in enumerate(correct_indices[:9]):
  plt.subplot(3, 3, i + 1)
  plt.imshow(X_test[incorrect].reshape(32,32,3), interpolation='none')
  plt.title(\"Predicted {}, Class {}\".format(y_class[incorrect], y_test[incorrect]))
plt.figure()
plt.show()

In [0]:
nb_classes = 10
class_name = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}
def draw_img(i):
  im = X_train[i]
  c = y_train[i]
  plt.imshow(im)
  plt.title("Class %d (%s)" % (c, class_name[c]))
  draw_img(11231)
  draw_img(231)
  draw_img(123)
  draw_img(321)
            

In [0]:
!ls

In [0]:
model.summary()

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import keras.backend as K
import cv2
img_path_list = ['/content/horse.png','/content/bird.png','/content/cat.png','/content/frog.png']
for i in range(len(img_path_list)):
    img_path = img_path_list[i]
    img = image.load_img(img_path, target_size=(32, 32))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
      preds = model.predict(x)
      class_idx = np.argmax(preds[0])
      class_output = model.output[:, class_idx]
  last_conv_layer = model.get_layer("zero_padding2d_11")
  grads = K.gradients(class_output, last_conv_layer.output)[0]
  pooled_grads = K.mean(grads, axis=(0, 1, 2))
  iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
  pooled_grads_value, conv_layer_output_value = iterate([x])
  for i in range(10):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
  heatmap = np.mean(conv_layer_output_value, axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  from google.colab.patches import cv2_imshow
  img = cv2.imread(img_path)
  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
  heatmap = np.uint8(255 * heatmap)
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
  superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
  superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
  numpy_horizontal_concat = np.concatenate((img, superimposed_img), axis=1)
  cv2_imshow(numpy_horizontal_concat)

In [0]:
#### Confusion Matrix (Add elements along the diagonal = No of correctly classified images)
from sklearn.metrics import classification_report, confusion_matrix
for ix in range(10):
   print(ix, confusion_matrix(np.argmax(Y_test,axis=1),y_class)[ix].sum())
cm = confusion_matrix(np.argmax(Y_test,axis=1),y_class)
print(cm)
# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd
df_cm = pd.DataFrame(cm, range(10), range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()